# Problem

Popular question and answer (qna) site -  stackoverflow (+ their sister sites) allows for download of monthly data dumps from https://archive.org/details/stackexchange.

With this data, can we classify the questions/answers based on

* Identify similar questions
* A particular question is associated with another question in terms of the next things to do or perhaps the pre-requisites?
* Predict the next question a user may ask based on this current search

The taxanomy could be a useful layout of the land for a student of the area.

# Schema

The schema for their data is located @ https://ia800500.us.archive.org/22/items/stackexchange/readme.txt.
    
Unfortunately, the data is dumped in an XML format and there is preliminary effort to convert that data into CSV format. We have written a converter (convert2csv.py) for the tables of interest.

# Conversion from XML to CSV

Run python convert2csv.py to convert each of the xml files to their CSV equivalents. For columns/attributes which contain textual data, the converter encodes them with base64 encoding so that handling of quotes and special characters (separators) is avoided. 

When the data is read back into the dataframe, the corresponding decode (from base64) needs to happen. The converter also creates a sample file of 100 rows for each xml data dump converted.

In [493]:
#imports
import pandas as pd
import math
import re
import gensim
from gensim import corpora, models,similarities
from gensim.models import word2vec, doc2vec
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


SAMPLE_SIZE = 20000

In [515]:
posts = pd.read_csv('data/posts.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna(subset=['Body','Title'])
posts['Tags'] = posts['Tags'].apply(lambda x : x.replace('<',' ').replace('>',' '))
posts[['Body','Title', 'Tags']].head(2)


,Body,Title,Tags
0,when should i use can when should i use could...,when do i use can or could,word-choice tenses politeness subjunctive-...
1,doesn t quint mean five what does that h...,where does the quint in quintessential com...,etymology


In [453]:
#comments = pd.read_csv('data/comments.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna()
#comments[['Score','Text']].head(5)

In [454]:
posthistory = pd.read_csv('data/posthistory.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna(subset=['Text'])
posthistory[['Text']].head(5)

,Text
0,when could i use can or when can i use could ...
1,when do i could can could
3,doesn t quint mean five what does that hav...
4,where does the quint in quintessential come ...
6,which is the correct use of these two words an...


In [455]:
users = pd.read_csv('data/users.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna(subset=['AboutMe','Location'])
users[['Location','AboutMe']].head(5)

,Location,AboutMe
0,on the server farm,hi i m not really a person i m a background ...
1,corvallis or,developer on the stack overflow team find me...
2,new york ny,developer on the stack overflow team was dubb...
3,raleigh nc,i design stuff for stack exchange also a prof...
4,california,i slip my front end into the back end and the...


## Further cleansing

* Remove (html) tags & carriage returns from the Text field
* Remove stop words (pick up the nltk stop words)
* Use PorterStemmer to stem words

In [456]:
#global
p_stemmer = PorterStemmer()
stop_words = stopwords.words('english')
stop_words.append('use')
#print(stop_words)

In [457]:
class SentenceTokens():
    def __init__(self,df,field):
        self.field = field
        self.df = df
    
    def __iter__(self):
      for index, row in self.df.iterrows():
         raw_sentence = row[self.field]
         raw_tokens = filter(None, re.split("[ ]+",raw_sentence))
         #stem_tokens = [p_stemmer.stem(tok) for tok in raw_tokens]
         yield [tok for tok in raw_tokens if not tok in stop_words and len(tok) > 1 ]

#all posts is a list of (list of tokens). The inner list of tokens is created once for each post
allposts = SentenceTokens(posts,'Title')


class LabeledLineSentence(object):
    def __init__(self,df,field,tag):
        self.df = df
        self.field = field
        self.tag = tag

    def __iter__(self):
        for index, row in self.df.iterrows():
           tags = ['T_' + tag for tag in row[self.tag].split()]
           yield doc2vec.TaggedDocument(words=row[self.field].split(),tags=tags)

lablines = LabeledLineSentence(posts,'Title','Tags')
#print([p for p in allposts])

In [458]:
#How frequently each term occurs within each document? We construct a document-term matrix.
dictionary = corpora.Dictionary(allposts)
#print(dictionary.token2id) #maps ids to tokens

In [504]:
#bag of words
corpus = [dictionary.doc2bow(text) for text in allposts]
#corpus, is a list of vectors equal to the number of documents. 
#In each document vector is a series of tuples. 
#print(corpus[0])
#print(len(corpus))
#print(corpus)
#(term ID, term frequency) pairs

In [494]:
# convert tokenized documents to vectors
new_doc = "bunary randam trees unorder"
new_vec = dictionary.doc2bow(new_doc.lower().split())

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
vec_lsi = lsi[new_vec] # convert the query to LSI space

index = similarities.MatrixSimilarity(lsi[corpus]) 

In [520]:
sims = index[vec_lsi] # perform a similarity query against the corpus
#print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples
sims = sorted(enumerate(sims), key=lambda item: -item[1])
#print(sims) # print sorted (document number, similarity score) 2-tuples
sims_list = list(enumerate(sims))
for i in range(0, len(sims_list)):
    #print(sims_list[i])
    docid = sims_list[i][1][0]
    print(str(sims_list[i][1][1] * 100) + " : " + posts.iloc[docid]['Title'])

0.0 : when do i use  can  or  could  
0.0 : where does the  quint  in  quintessential  come from 
0.0 : when should i use  shall  versus  will  
0.0 : when did  while  and  whilst  become interchangeable 
0.0 :  may     might   what s the right context 
0.0 : is it appropriate to use short form of  have    ve  when it means possession 
0.0 : which words in a title should be capitalized 
0.0 : when is it appropriate to end a sentence in a preposition 
0.0 : where did the term  ok okay  come from 
0.0 : what is the proper plural of the word  freshman  
0.0 : where did the singular  innings  come from 
0.0 : are  betwixt    trebble   etc   acceptable in american english 
0.0 : is it ever acceptable for a period to come after a quote at the end of a sentence 
0.0 :  what s wrong in with this question  
0.0 : is there a correct gender neutral  singular pronoun   his  versus  her  versus  their    
0.0 : can a word be contracted twice  e g   i ven t   
0.0 : what   s the rule for using    wh

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=30)

#num_topics: required. An LDA model requires the user to determine how many topics should be generated. 
#id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.
#passes: optional. The number of laps the model will take through corpus.

In [461]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [462]:
@interact(num_topics=5, num_words=3)
def understand(num_topics, num_words):
    return ldamodel.print_topics(num_topics, num_words)
    
#Each generated topic is separated by a comma.
#Within each topic are the three most probable words to appear in that topic.

[(0, u'0.026*pronunciation + 0.014*vs + 0.013*verb'),
 (1, u'0.072*sentence + 0.019*word + 0.016*end'),
 (9, u'0.080*vs + 0.031*plural + 0.018*singular'),
 (3, u'0.062*english + 0.048*words + 0.034*word'),
 (7, u'0.122*mean + 0.011*phrase + 0.011*names')]

In [463]:
pyLDAvis.gensim.prepare(ldamodel,corpus,dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      11.163456        1       1 -0.096942  0.046123
1      10.716266        1       2 -0.062421 -0.112720
2      10.590291        1       3 -0.072369  0.042732
4      10.565406        1       4  0.230919 -0.023053
8      10.240344        1       5 -0.010327  0.039813
9       9.961510        1       6  0.151080 -0.038283
6       9.619657        1       7 -0.121606 -0.072152
5       9.265786        1       8 -0.089243 -0.102646
7       8.949452        1       9 -0.010660  0.258587
0       8.927832        1      10  0.081568 -0.038401, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
3135  Default  263.000000           mean  263.000000  30.0000  30.0000
3831  Default  236.000000     difference  236.000000  29.0000  29.0000
3304  Default  165.000000       sentence  165.000000  28.0000  28.0000
1418  Default  249.000000             vs  249.000000  27.0000  27.0000
5932  Default  212.000000        english  212.000000  26.0000  26.0000
5673  Default  210.000000        meaning  210.000000  25.0000  25.0000
5946  Default  133.000000          usage  133.000000  24.0000  24.0000
5853  Default  128.000000          words  128.000000  23.0000  23.0000
3557  Default  210.000000        correct  210.000000  22.0000  22.0000
4455  Default   86.000000           like   86.000000  21.0000  21.0000
5025  Default   90.000000           term   90.000000  20.0000  20.0000
5793  Default   66.000000         plural   66.000000  19.0000  19.0000
1626  Default   75.000000         versus   75.000000  18.0000  18.0000
5646  Default   61.000000       question   61.000000  17.0000  17.0000
1585  Default  366.000000           word  366.000000  16.0000  16.0000
4140  Default   49.000000  pronunciation   49.000000  15.0000  15.0000
3896  Default   50.000000      different   50.000000  14.0000  14.0000
5739  Default   65.000000      something   65.000000  13.0000  13.0000
4571  Default   72.000000          would   72.000000  12.0000  12.0000
3268  Default  116.000000            say  116.000000  11.0000  11.0000
5053  Default   87.000000         origin   87.000000  10.0000  10.0000
2557  Default  120.000000           used  120.000000   9.0000   9.0000
1236  Default   39.000000       singular   39.000000   8.0000   8.0000
4466  Default   50.000000         proper   50.000000   7.0000   7.0000
4596  Default   36.000000            end   36.000000   6.0000   6.0000
4006  Default   40.000000         common   40.000000   5.0000   5.0000
2191  Default   65.000000          using   65.000000   4.0000   4.0000
4151  Default   31.000000         single   31.000000   3.0000   3.0000
2414  Default   33.000000       american   33.000000   2.0000   2.0000
2040  Default   32.000000        british   32.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
397   Topic10    5.240678          leave    5.941616   2.2905  -5.8989
253   Topic10    5.120567         played    5.821574   2.2877  -5.9221
2753  Topic10    5.038693     university    5.739607   2.2858  -5.9382
2001  Topic10    4.905932           half    5.606874   2.2824  -5.9649
6026  Topic10    4.708252      auxiliary    5.409156   2.2772  -6.0060
4098  Topic10    4.588202            bit    5.289114   2.2738  -6.0318
2730  Topic10   16.403662       spelling   21.278331   2.1558  -4.7578
3221  Topic10   13.554084            bad   17.594819   2.1551  -4.9486
168   Topic10   13.084378         object   18.764701   2.0554  -4.9839
1520  Topic10   13.675843           rule   20.652918   2.0038  -4.9397
289   Topic10   10.829920       possible   15.752503   2.0413  -5.1730
4261  Topic10   13.057565            day   20.717371   1.9544  -4.9860
3755  Topic10   15.895505      pronounce   29.492167   1.7979  -4.7893
3624  Topic10   12.603716        synonym   

In [464]:
#wordmodel = word2vec.Word2Vec(allposts)
#wordmodel.init_sims(replace=True)
#docmodel = doc2vec.Doc2Vec(lablines, min_count=1)


docmodel = doc2vec.Doc2Vec(alpha=0.025, min_alpha=0.025)
docmodel.build_vocab(lablines)
for epoch in range(10):
    docmodel.train(lablines)
    docmodel.alpha -= 0.002  # decrease the learning rate
    docmodel.min_alpha = docmodel.alpha  # fix the learning rate, no decay



In [491]:
from pprint import pprint
def showsimilar(q):
    norm_input = [tok for tok in q.split() if not tok in stop_words and len(tok) > 1 ]
    input_vec = docmodel.infer_vector(norm_input)
    similar_vecs = docmodel.docvecs.most_similar(positive=[input_vec])
    similartags = [s[0] for s in similar_vecs]

    def matches(data,col,values):
        match = False
        for v in values:
            match =  data[col].str.contains(v.replace('T_',''))
            if match is True:
                return match
        return match

    similarposts = posts[matches(posts,'Tags',similartags)]
    pprint(similarposts[['Title']])


In [492]:
from IPython.display import display
from ipywidgets import widgets 
import warnings

warnings.filterwarnings('error')

def handler(sender):
    showsimilar(text.value)
try:
    text = widgets.Text()    
except DeprecationWarning:
    text.on_submit(handler)
display(text)    

## References

* https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
* LDA Viz - http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf
* This dashboard @ https://github.com/dhruvaray/soml
    